## Convolutional neural networks with low-rank regularization
This notebook shows an example of SVD-based low-rank factorization for convolutional layers in a neural network. We will use VGG16 during this example.

In [1]:
import tensorflow as tf
from lowrank import *

model = tf.keras.models.load_model('../vgg16', compile=False)
model = model.layers[-1]

The rank choosing in this process is manual, and for VGG16 we'll use the same values used in the paper of Tai *et al.*, as those are the ones I got the best [speed x accuracy] results with. For the other networks I tested, a criteria proposed by Tai *et al.* was to use a rank that "explained" 95% of a layer's variance (PCA-style), but I'm using the same ranks as in CP-decomposition as I got better results with those.

In [15]:
#mod_name = 'mobilenet'
#mod_name = 'resnet50'
mod_name = 'vgg16'

if mod_name == 'mobilenet':
    ranks = {'block_16_project': 181, 'Conv_1': 180}
elif mod_name == 'resnet50':
    ranks = {'conv1_conv': 5, 'conv2_block1_1_conv': 5, 'conv2_block1_2_conv': 5, 'conv2_block1_0_conv': 5, 
             'conv2_block1_3_conv': 5, 'conv2_block2_1_conv': 5, 'conv2_block2_2_conv': 5, 
             'conv2_block2_3_conv': 5, 'conv2_block3_1_conv': 5, 'conv2_block3_2_conv': 5, 
             'conv2_block3_3_conv': 5, 'conv3_block1_1_conv': 5, 'conv3_block1_2_conv': 5, 
             'conv3_block1_0_conv': 25, 'conv3_block1_3_conv': 12, 'conv3_block2_1_conv': 5, 
             'conv3_block2_2_conv': 9, 'conv3_block2_3_conv': 5, 'conv3_block3_1_conv': 5, 
             'conv3_block3_2_conv': 5, 'conv3_block3_3_conv': 5, 'conv3_block4_1_conv': 5, 
             'conv3_block4_2_conv': 16, 'conv3_block4_3_conv': 5, 'conv4_block1_1_conv': 13, 
             'conv4_block1_2_conv': 28, 'conv4_block1_0_conv': 119, 'conv4_block1_3_conv': 102, 
             'conv4_block2_1_conv': 92, 'conv4_block2_2_conv': 101, 'conv4_block2_3_conv': 78, 
             'conv4_block3_1_conv': 90, 'conv4_block3_2_conv': 93, 'conv4_block3_3_conv': 71, 
             'conv4_block4_1_conv': 100, 'conv4_block4_2_conv': 108, 'conv4_block4_3_conv': 92, 
             'conv4_block5_1_conv': 124, 'conv4_block5_2_conv': 114, 'conv4_block5_3_conv': 96, 
             'conv4_block6_1_conv': 145, 'conv4_block6_2_conv': 168, 'conv4_block6_3_conv': 148, 
             'conv5_block1_1_conv': 215, 'conv5_block1_2_conv': 240, 'conv5_block1_0_conv': 353, 
             'conv5_block1_3_conv': 345, 'conv5_block2_1_conv': 368, 'conv5_block2_2_conv': 357, 
             'conv5_block2_3_conv': 346, 'conv5_block3_1_conv': 409, 'conv5_block3_2_conv': 414, 
             'conv5_block3_3_conv': 408}
elif mod_name == 'vgg16':
    ranks = {'block1_conv1':5, 'block1_conv2':24, 'block2_conv1':48, 'block2_conv2':48, 
             'block3_conv1':64, 'block3_conv2':128, 'block3_conv3':160, 'block4_conv1':192, 
             'block4_conv2':192, 'block4_conv3':256, 'block5_conv1':320, 'block5_conv2':320, 
             'block5_conv3':320}


As the kernel of a conv. layer is a 4D tensor, we "matricize" it in order to apply SVD directly. All factorized layers are tested with the formula `K < dCN/(C+N)` (K, d, C & N being the rank, kernel size, input & output num. of channels, respectively) to check if the optimization would actually accelerate the layer. If it wouldn't, the layer is left untouched.

In [ ]:
model_lr = LR_optimize(model, 'manual', ranks)
model_lr.summary()

netin = keras.Input((224,224,3), dtype='uint8')
x = tf.cast(netin, 'float32')
if mod_name == 'mobilenet':
    x = keras.applications.mobilenet_v2.preprocess_input(x) 
elif mod_name == 'resnet50':
    x = tf.nn.bias_add(x, [-123.68, -116.779, -103.939])
elif mod_name == 'vgg16'
    x = keras.applications.imagenet_utils.preprocess_input(x, mode='torch')
x = model_lr(x)
model_lr = keras.Model(inputs=netin, outputs=x)

Finally, we add the preprocessing layers and save the model.

In [ ]:
model_lr.save('%s_LR' % mod_name)